In [1]:
import pandas as pd
import numpy as np


In [2]:
df= pd.read_csv('../Cleaned_Data/cleaned_data.csv')

In [3]:
df

,title,content,type
0,5,"Sách rất hay. Dựa trên nghiên cứu, và có hướng...",0
1,5,Nội dung sách quá hay Rất đơn giản và dễ áp dụ...,0
2,5,Quyển này hay mà tác giả viết rất dễ đọc nha n...,0
3,5,Dạo này thói quen xấu lại bủa vây nên tôi quyế...,0
4,5,"Tiki giao thần tốc, mình đặt 1h sáng giao chiề...",0
...,...,...,...
7968,4,Ship về bị chảy ra ngoài,25
7969,4,chưa dùng sp nên chưa đánh giá dc,25
7970,4,bọt ít k sạch dầu mỡ,25
7971,4,Giá hơi cao,25


# Hệ thống Dựa trên Luật (Rule-Based Systems) cho việc phân loại khía cạnh

Trong phần này, chúng ta sẽ triển khai một hệ thống dựa trên luật đơn giản để phân loại các đánh giá theo các khía cạnh khác nhau:
- Vận chuyển
- Chất lượng
- Giá cả
- Chăm sóc khách hàng (CSKH)

Mỗi luật sẽ kiểm tra sự xuất hiện của các từ khóa cụ thể trong nội dung đánh giá.

In [4]:
van_chuyen_keywords = [
    'giao hàng', 'ship', 'shipper', 'vận chuyển', 'giao', 'nhận hàng', 'đóng gói',
    'thời gian giao', 'giao nhanh', 'giao chậm', 'phí ship', 'thần tốc','Gói hàng'
]

chat_luong_keywords = [
    'chất lượng', 'bền', 'hỏng', 'tốt', 'xấu', 'đểu', 'kém', 'chắc chắn',
    'dễ vỡ', 'hư', 'dùng tốt', 'sản phẩm tốt', 'hoạt động', 'dùng', 'chất lượng kém','Nội dung','hay','đẹp','xịn','độc','lạ',
    'đẹp trai', 'đẹp gái', 'mới', 'cũ', 'hàng chính hãng', 'hàng giả', 'hàng nhái','Tuyệt vời'
]

gia_ca_keywords = [
    'giá', 'đắt', 'rẻ', 'khuyến mãi', 'hợp lý', 'giá cả', 'giá trị', 'tiền',
    'đáng tiền', 'không đáng tiền', 'sale', 'giảm giá', 'rẻ hơn', 'đắt hơn'
]

cskh_keywords = [
    'nhân viên', 'tư vấn', 'hỗ trợ', 'chăm sóc', 'thái độ', 'phục vụ',
    'nhiệt tình', 'phản hồi', 'chăm sóc khách hàng', 'khách hàng', 'dịch vụ'
]

In [5]:
def contains_keyword(text, keywords):
    if isinstance(text, str):
        text = text.lower()
        for keyword in keywords:
            if keyword in text:
                return True
    return False

def classify_aspect(text):
    aspects = []
    
    if contains_keyword(text, van_chuyen_keywords):
        aspects.append('ship')
        
    if contains_keyword(text, chat_luong_keywords):
        aspects.append('quality')
        
    if contains_keyword(text, gia_ca_keywords):
        aspects.append('price')
        
    if contains_keyword(text, cskh_keywords):
        aspects.append('cskh')
        
    if not aspects:
        return 'other'
    
    return ', '.join(aspects)

In [6]:
df['aspect'] = df['content'].apply(classify_aspect)

print("Phân phối các khía cạnh:")
aspect_counts = df['aspect'].value_counts()
print(aspect_counts)

Phân phối các khía cạnh:
aspect
quality                       2856
ship, quality                 1616
other                         1504
ship                           724
quality, price                 491
ship, quality, price           293
price                          153
ship, quality, cskh             96
quality, cskh                   64
ship, price                     60
ship, quality, price, cskh      37
ship, cskh                      28
quality, price, cskh            24
cskh                            17
price, cskh                      5
ship, price, cskh                5
Name: count, dtype: int64


In [7]:
# Xem một số ví dụ cho mỗi khía cạnh
for aspect in aspect_counts.index[:5]:
    print(f"\n\n--- Ví dụ cho khía cạnh: {aspect} ---")
    sample = df[df['aspect'] == aspect].sample(min(3, len(df[df['aspect'] == aspect])))
    for i, (_, row) in enumerate(sample.iterrows()):
        print(f"Đánh giá {i+1}: {row['content']}")



--- Ví dụ cho khía cạnh: quality ---
Đánh giá 1: Truyện hay, lôi cuốn và ý nghĩa lắm ạ.
Đánh giá 2: Áo đẹp nhưng hơi rộng tí
Đánh giá 3: Sách không đc bao kĩ làm ảnh hưởng bìa sách


--- Ví dụ cho khía cạnh: ship, quality ---
Đánh giá 1: Ship nhanh, sách hay lắm
Đánh giá 2: giao nhanh. sách đẹp. 5☆
Đánh giá 3: sách đẹp, gói hàng tốt, giao hàng nhanh


--- Ví dụ cho khía cạnh: other ---
Đánh giá 1: Dép nhẹ, đi êm
Đánh giá 2: Tai nghe khá ổn nghe lâu thì hơi nên đổi sang núm mềm
Đánh giá 3: Rất hài lòng!


--- Ví dụ cho khía cạnh: ship ---
Đánh giá 1: giao hàng nhanh đóng gói cẩn thận
Đánh giá 2: Giao nhanh ạ!
Đánh giá 3: Đóng gói cẩn thận giao nhanh


--- Ví dụ cho khía cạnh: quality, price ---
Đánh giá 1: Rẻ hơn.nhưng vẫn sợ hàng kém chất lượng
Đánh giá 2: Bất ngờ rẻ mà tốt
Đánh giá 3: Nhìn cũng được, chưa dùng nên chưa có đánh giá về chất lượng


In [8]:
output_path = '../Labeled_Data/Simple_Label.csv'
df.to_csv(output_path, index=False)
df.to_excel("../Labeled_Data/Simple_Label.xlsx", index=False)

print(f"Đã lưu dữ liệu đã gán nhãn vào {output_path}")

Đã lưu dữ liệu đã gán nhãn vào ../Labeled_Data/Simple_Label.csv


In [9]:
other_df = df[df['aspect'] == 'other']

print(f"Số lượng đánh giá với nhãn 'other': {len(other_df)}")

print("\nMột số ví dụ đánh giá với nhãn 'other':")
print(other_df.sample(5)['content'])

other_output_path = '../Labeled_Data/Other_Label.csv'
other_df.to_csv(other_output_path, index=False)
other_df.to_excel("../Labeled_Data/Other_Label.xlsx", index=False)
print(f"\nĐã lưu các đánh giá có nhãn 'other' vào {other_output_path}")

Số lượng đánh giá với nhãn 'other': 1504

Một số ví dụ đánh giá với nhãn 'other':
4662       Sách review nhiều mua đọc thêm
7512          túi méo mó quá ko vuông vắn
1307    Truyện đọc khá logic và nhẹ nhàng
7388                       Xinh lắm luôn!
3230        nội dung sách hơi bị lan man.
Name: content, dtype: object

Đã lưu các đánh giá có nhãn 'other' vào ../Labeled_Data/Other_Label.csv
